[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klanita/PoincareMSA/blob/master/PoincareMSA_colab.ipynb)

<img src="https://github.com/klanita/PoincareMSA/blob/master/.github/PoincareMSA_small_logo.png?raw=true" height="100" style="height:100px;margin-left: 0px;">


# Poincaré maps for visualization of large protein famillies

PoincareMSA builds a projection of protein multiple sequence alignemnt (MSA) on a Poincaré disk. The proximity of the points to the disk center corresponds to their hierarchy and correlates with the proximity of the proteins to the root of the phylogenetic tree. Thus, must central point often correspond to the ancestor proteins and protein located close to the border to the leaves of phylogenetic tree.

# Notebook initialization

### Load dependencies

In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#Import visualization functions
from scripts.visualize_projection.pplots_new import read_embeddings, plot_embedding, plot_embedding_interactive, rotate, get_colors
%matplotlib inline

#Create optional variables
path_annotation = ""

# Data import

In [2]:
# OPTIONS =================================================
mfasta = "new_examples/thioredoxins/thioredoxins.mfasta"
path_annotation = "new_examples/thioredoxins/thioredoxin_annotation.csv"
#==========================================================

#Check files
#mfasta
nb_seq = 0
if os.path.isfile(mfasta):
    with open(mfasta, "r") as f_in:
        for line in f_in:
            if line[0] == ">":
                nb_seq += 1
    print(f"\nNumber of sequences found: {nb_seq}.")
else:
    print(f"File {mfasta} not found.")

#Annotations
if os.path.isfile(path_annotation):
    try:
        df_annotation = pd.read_csv(path_annotation)
        if len(df_annotation) != nb_seq:
            raise ValueError("Annotation file doesn't match the .mfasta file length.")
    except:
        raise ValueError("Annotation file is not in .csv format.")
    print("\nAnnotation file correctly loaded.")
    annotation_names = list(df_annotation.columns)
    print(f"{len(annotation_names)} annotations found: {annotation_names}.")
else:
    print(f"File {path_annotation} not found")


Number of sequences found: 210.

Annotation file correctly loaded.
5 annotations found: ['Unnamed: 0', 'Species', 'Kingdom', 'Family', 'Color_eukaryota'].


# Settings

In [3]:
# OPTIONS =================================================
# Job name
out_name = "thioredoxins_data"
#----------------------------------------------------------
# Threshold for filtering gapped positions
gapth = 0.9 
#----------------------------------------------------------
# Projection parameters
knn = 5
gamma = 2
sigma = 1
cospca = 0
batchs = 4
epochs = 1000
seed = 0
#==========================================================

# Data preparation and data projection using Poincaré disk

In [4]:
# 1. Data preparation
# Data preparation consists in `.mfasta` cleaning according to a gap threshold and translation of each sequence to the PSSM profile.
print("1. Data preparation")
prep_parameters = "scripts/prepare_data" + " " + mfasta + " " + out_name + " " + out_name + " " + str(gapth)
bash_projection = "bash scripts/prepare_data/create_projection.sh " + prep_parameters
!{bash_projection}

print("\n2. Data projection using Poincaré disk")
# 2. Data projection using Poincaré disk
#This step creates a projection of encoded sequences to a Poincaré disk.
bash_pm = "python3 "+ "scripts/build_poincare_map/main.py --input_path " + out_name + "/fasta" + str(gapth) + " --output_path " + out_name + "/projections/ --gamma "+ str(gamma) +" --pca "+ str(cospca) + " --epochs "+ str(epochs) +" --seed "+ str(seed) + " --knn " + str(knn)
!{bash_pm}

print("\n3. Format data for visualization")
# 3. Format data for visualization
#Check that an annotation file was provided. Create a dummy one instead
if not path_annotation:
    df_annotation = pd.DataFrame(np.full(nb_seq, "-", dtype=object), columns=["default"])
    df_annotation.to_csv("dummy_annotation.csv", index=False)
    path_annotation = "dummy_annotation.csv"
    annotation_names = ["default"]

path_embedding = f"{out_name}/projections/PM{knn:1.0f}sigma={sigma:2.2f}gamma={gamma:2.2f}cosinepca={cospca:1.0f}_seed{seed:1.0f}.csv"
df_embedding = read_embeddings(path_embedding, path_annotation, withroot=False)

1. Data preparation
Input file: new_examples/thioredoxins/thioredoxins.mfasta
Name of the protein family: thioredoxins
1 X aa replaced by gaps in 210 sequences
filter_gaps finished for new_examples/thioredoxins/thioredoxins.mfasta
mfasta2fasta finished for thioredoxins_data/thioredoxins_data.clean0.9.mfasta
1 X aa replaced by gaps in 210 sequences

2. Data projection using Poincaré disk
CUDA: True
210 proteins found in folder thioredoxins_data/fasta0.9.
No root detected
Prepare data: tensor construction
Prepare data: successfully terminated
Computing laplacian...
Laplacian computed in 0.05 sec
Computing RFA...
RFA computed in 0.00 sec
Starting training...
loss: 0.53214: 100%|████████████████████████| 1000/1000 [07:01<00:00,  2.38it/s]
PM computed in 421.02 sec

loss = 5.321e-01
time = 7.020 min

3. Format data for visualization


# Projection visualization

In [5]:
# Construction of custom color palette 
# class species from thioredoxin annotation file into different kingdom 
bacteria_specie = ["Bacteria", "LBCA", "LGPCA",  "LPBCA", "Salmonella", "Shigella", "Escherichia", "Acidobacteria", "Deinococcus", "Thermus", 
                    "Solibacter", "Helicobacter", "Geobacter", "Wolinella", "Neisseria", "Bordetella", "Brucella", "Rickettsia", "Campylobacter",
                   "Thiobacillus", "Burkholderia", "Vibrio",  "Desulfovibrio", "Bdellovibrio", "Yersinia", "Agrobacterium", "Actinobacteria", "Thermobifida",
                   "Flavobacterium","Geobacterium", "Chloroflexus", "Aquifex", "Enterococcus","Chlamydia", "Chlamydophila", "Listeria", "Lactobacillus",
                   "Geobacillus",  "Chlorobium",  "Rhodopirrelula", "Clostridium", "Hyperthermus", "Porphyromonas", "Streptomyces", 
                   "Syneschocystis", "Nostoc", "Thermosynechococus", "Prochlorococcus", "Nostoc1", "Sinorhizobium", "Pseudomonas", "Bacteroides", "Staphylococcus", 
                   "Thermoanaerobacter","Bacillus", "Mycobacterium", "Streptococcus", "Dehalococcoides", "Synechocystis", "Corynebacterium", "Thermosynechococcus",
                   "Rhodopirellula"]
archae_specie = ["Archaea", "LACA", "Aeropyrum","Thermofilum","Caldivirga","Sulfolobus","Haloquadratum","Haloarcula", "Thermoplasma", "Hyperthermus",
                 "Natronomonas","Methanocorpusculum", "Methanococcus", "Halobacterium", "Picrophilus","Methanospirillum","Staphylothermus",
                  "Methanosaeta", "Metallosphaera","Methanococcoides", "Candidatus","Archaeoglobus"]
eukaryota_animal= ["Eukaryota", "LECA", "LAFCA", "Ovis","Bos", "Mus","Rattus","Rabit", "Human", "Ponab","Macmu","Ornithorhynchus","Gallus","Equus",
                    "Theileria","Danio","Tetraodon","Xenopus","Ictalurus","Ophiophagus","Callithrix","Monodelphis","Geocy","Sus","Melopsittacus", # animal
                    "Monosiga", # choanoflagellata
                    "Graphocephala", "Tribolium", "Apis", "Bombyx","Litopenaeus","Drosophila", # insecta
                    "Entamoeba", "Plasmodium","Cryptosporidium", "Dictyostelium"]  # unicellular parasite  
eukaryota_viridiplantae = ["Helicosporidium", "Ostreococcus", "Fagopyrum"]
eukaryota_mitochondrion = ["Bovin Mitochondrio", "Homo Mitochondrion",  "Rattus Mitochondrio", "Mus Mitochondrion"] 
eukaryota_chloroplast = ["Brana Chloroplast", "Pisum Chloroplast",  "Wheat Chloroplast", "Pea Chloroplast", "Spiol Chloroplast", "Arabidopsis"] 
eukaryota_fungi = ["Pichia", "Candida", "Aspergillus", "Kluyveromyces", "Saccharomyces","Schizosaccharomyces","Neosartorya"] 

# Construction of palette 
trx_palette = {"OTHERS" : "#c7c7c7", "EUKARYOTA" : "#31955d", "CHLOROPLAST" : "#06f9d4", "MITOCHONDRION" : "#3ff100" , "root": "#000000", "AECA" : "#e7e53c",
               "Zea": "#c7c7c7", "Vitis":"#c7c7c7", "Limonium": "#c7c7c7"} 
trx_palette.update(dict.fromkeys(eukaryota_animal,"#21e548"))
trx_palette.update(dict.fromkeys(eukaryota_viridiplantae,"#b5e521"))
trx_palette.update(dict.fromkeys(eukaryota_mitochondrion, "#61ff06"))
trx_palette.update(dict.fromkeys(eukaryota_chloroplast,"#06f9d4"))
trx_palette.update(dict.fromkeys(eukaryota_fungi,"#1fb995"))
trx_palette.update(dict.fromkeys(bacteria_specie,"#34b8e7"))
trx_palette.update(dict.fromkeys(archae_specie, "#ff0000"))

In [7]:
# OPTIONS =================================================
title = "PM projection of thioredoxins colored by kingdom of life"                                  
#----------------------------------------------------------
# Select the coloring from annotation .csv file:
labels_name = "Kingdom"
# Select classes to label among the "labels_name" column (comma separated list):
labels_text = ""
show_text = False
#----------------------------------------------------------
# Use a custom color palette:
color_palette = trx_palette
use_custom_palette = True
#==========================================================


#Check projection visualization parameters
#Labels name
if labels_name == "" and path_annotation == "dummy_annotation.csv":
    labels_name = "default"
elif labels_name == "":
    labels_name = None
elif labels_name not in annotation_names:
    raise NameError(f"labels_name {labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}")

#Labels text
try:
    labels_text = [s.strip() for s in labels_text.split(",")]
except:
    print("Error: label_text field is not a valid list.")

#Convert labels_text to labels_name dtype
if labels_name and labels_name != "default":
    try:
        labels_text_dtype = df_annotation[labels_name].dtypes
        labels_text = list(np.array(labels_text).astype(labels_text_dtype))
    except:
        raise TypeError(f'"labels_text" is not compatible with {labels_name}" data format ({labels_text_dtype}).')

if not use_custom_palette:
    color_palette = None

#Plot graph
fig = plot_embedding_interactive(df_embedding, 
                                 labels_name = labels_name,
                                 show_text = show_text,
                                 labels_text = labels_text,
                                 color_palette = color_palette, 
                                 title = title, 
                                 fontsize = 11)
fig.show()

In [8]:
# OPTIONS =================================================
title = "PM projection of thioredoxins colored by species"
#----------------------------------------------------------
# Select the coloring from annotation .csv file:
labels_name = "Species"
# Select classes to label among the "labels_name" column (comma separated list):
labels_text = ""
show_text = False
#----------------------------------------------------------
# Use a custom color palette:
color_palette = trx_palette
use_custom_palette = True
#==========================================================


#Check projection visualization parameters
#Labels name
if labels_name == "" and path_annotation == "dummy_annotation.csv":
    labels_name = "default"
elif labels_name == "":
    labels_name = None
elif labels_name not in annotation_names:
    raise NameError(f"labels_name {labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}")

#Labels text
if labels_text:
    try:
        labels_text = [s.strip() for s in labels_text.split(",")]
    except:
        print("Error: label_text field is not a valid list.")
else:
    labels_text = [""]

#Convert labels_text to labels_name dtype
if labels_name and labels_name != "default" and labels_text != [""]:
    try:
        labels_text_dtype = df_annotation[labels_name].dtypes
        labels_text = list(np.array(labels_text).astype(labels_text_dtype))
    except:
        raise TypeError(f'"labels_text" is not compatible with {labels_name}" data format ({labels_text_dtype}).')

if not use_custom_palette:
    color_palette = None

#Plot graph
fig = plot_embedding_interactive(df_embedding, 
                                 labels_name = labels_name,
                                 show_text = show_text,
                                 labels_text = labels_text,
                                 color_palette = color_palette, 
                                 title = title, 
                                 fontsize = 11)
fig.show()

### Save plot to file

In [8]:
# OPTIONS =================================================
output_name = "fig1"
output_format = "png" #Format availables: ["png", "html", "pdf", "svg"]
#==========================================================


if output_format != "html":
    fig.write_image(f"{output_name}.{output_format}", engine="kaleido")
else:
    fig.write_html(f"{output_name}.{output_format}")